In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
!nvidia-smi

Wed Apr  3 05:25:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   40C    P0             231W / 400W |  13864MiB / 40960MiB |     92%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### To reproduce word2vec classification, you need to clone the pretrained model w2v first with below command

In [2]:
#!git clone https://github.com/deryrahman/word2vec-bahasa-indonesia.git

In [2]:
%cd word2vec-bahasa-indonesia
!dir

/workspace/Tugas Akhir/word2vec-bahasa-indonesia
LICENSE    data        model	       requirements.txt
README.md  example.py  pretrained-w2v  train.py


In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 23.1 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 54.7 MB/s eta 0:00:01
  Created wheel for gensim: filename=gensim-3.6.0-cp38-cp38-linux_x86_64.whl size=24812852 sha256=cb184ea281a5e2a71f54dac7fb1b66a25acc3450acde7810f263c9882c28bca1
  Stored in directory: /tmp/pip-ephem-wheel-cache-15ow2_7b/wheels/d6/5b/91/658efb08f9d790987e1ef3795e62e129868afb0fde409baac5
Successfully built gensim


In [4]:
#!mkdir pretrained-w2v
%cd pretrained-w2v

/workspace/Tugas Akhir/word2vec-bahasa-indonesia/pretrained-w2v


In [5]:
!pwd

/workspace/Tugas Akhir/word2vec-bahasa-indonesia/pretrained-w2v


In [6]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim


w2v = Word2Vec.load("idwiki_word2vec_200.model")

In [8]:
def get_vect(token):
    try:
        return w2v.wv[token]
    except:
        try:
            return w2v.wv[token.lower()]
        except:
            return w2v.wv["raja"]-w2v.wv["raja"] #return zero vector if OOV

In [9]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
def get_vector_for_sentence(sentence, max_num_token):
    tokenized = [word_tokenize(sent) for sent in sentence.split()]
    num = 0
    temp_sum = get_vect(".")
    for token in tokenized:
        if num >= max_num_token:
            break
        temp_sum += get_vect(token[0].lower())
        num+=1
    if num == 0:
        return temp_sum
    else:
        return temp_sum/num

def get_vector_all_sentence_from_a_column(df, kolom, max_num_token = 256):
    ret = []
    ret1 = [] 
    num = 0
    for row in df['trunc_combination']:
        ret1.append(get_vector_for_sentence(row, max_num_token))
    for row in df[kolom]:
        try:
            ret.append(get_vector_for_sentence(row, max_num_token))
        except:
            ret.append(ret1[num])
        num+=1
    mini=1e10
    for row in ret:
        for sentence in row:
            mini = min(mini, sentence)
    res = []
    for row in ret:
        res.append(row-mini)
    return res

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
!pip install scikit-learn  -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [15]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

def get_eval(ypred, yact):
    p,r, f,s = precision_recall_fscore_support(yact,ypred, average='binary', pos_label=1)
    print(f"Nilai Precision = {p}")
    print(f"Nilai Recall = {r}")
    print(f"Nilai F1 = {f}")
    return p,r,f


def get_eksperiment(df_train, df_test, fitur, bencana):
    %cd ..
    dct_cls = {}
    print(f"Eksperimen pada bencana {bencana} dengan representasi {fitur}")
    y_train = []
    #print(list(df_train))
    for l in list(df_train['label']):
        if l =='Y':
            y_train.append(1)
        else:
            y_train.append(0)
    y_test = []
    for l in list(df_test['label']):
        if l =='Y':
            y_test.append(1)
        else:
            y_test.append(0)
    X_train = get_vector_all_sentence_from_a_column(df_train,fitur)
    X_test = get_vector_all_sentence_from_a_column(df_test,fitur)
    rf = RandomForestClassifier(max_depth=100, n_estimators=100)
    lr = LogisticRegression(solver='liblinear',max_iter = 100)
    mlp = MLPClassifier(random_state=1, max_iter=300)
    xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05,max_depth=3, random_state=0)

    print("XGBOOST")
    xgb.fit(X_train, y_train)
    y_pred_train = xgb.predict(X_train)
    y_pred_test = xgb.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p,r,f = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p
    dct_metric["rec"] = r
    dct_metric["f1"] = f 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["xgb"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Word2Vec/{bencana}_{fitur}_XGBOOST.csv', index=False)


    print("RANDOM FOREST")
    rf.fit(X_train, y_train)
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["rf"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Word2Vec/{bencana}_{fitur}_RF.csv', index=False)

    print("LOGISTIK REGRESSION")
    lr.fit(X_train, y_train)
    y_pred_train = lr.predict(X_train)
    y_pred_test = lr.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["lr"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Word2Vec/{bencana}_{fitur}_LR.csv', index=False)
    p+=p_l
    r+=r_l
    f+=f_l

    print("MLP")
    mlp.fit(X_train, y_train)
    y_pred_train = mlp.predict(X_train)
    y_pred_test = mlp.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["mlp"] = dct_metric
    print("########")
    p+=p_l
    r+=r_l
    f+=f_l
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Word2Vec/{bencana}_{fitur}_MLP.csv', index=False)

    print('Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah')
    print(f'Precision = {p/3}\nRecall = {r/3}\nF1 = {f/3}')
    %cd -
    return dct_cls


In [19]:
def get_f1_eks_praproses_bencana(bencana):
    lst_fitur = ["id","title_ori", "text_ori", "tc_ori", "filter_word_tc","filter_word_tc_window", "trunc_first_short", "trunc_last_short", "trunc_first_long", "trunc_last_long", "trunc_combination",  "textrank", "sum_indobart", "sum_liputan6","sum_pegasus","label"]
    #lst_fitur = ["textrank", "sum_indobart", "sum_liputan6","sum_pegasus","label"]
    df_train = pd.read_csv(f'{bencana}_train.csv')[lst_fitur]
    df_dev = pd.read_csv(f'{bencana}_dev.csv')[lst_fitur]
    df_test = pd.read_csv(f'{bencana}_test.csv')[lst_fitur]
    print(f"evaluasi pada bencana = {bencana}")
    dct_bencana={}
    dct_bencana[bencana] = {}
    for fitur in lst_fitur[1:-1]:
        dct_fitur = {}
        dct_bencana[bencana][fitur] = get_eksperiment(df_train, df_test, fitur, bencana)

        print('\n')

        print('************************************************************')


    print(dct_bencana)

In [16]:
!pwd

/workspace/Tugas Akhir/word2vec-bahasa-indonesia/pretrained-w2v


In [17]:
%cd ..
%cd ..
%cd dataset

/workspace/Tugas Akhir/word2vec-bahasa-indonesia
/workspace/Tugas Akhir
/workspace/Tugas Akhir/dataset


In [20]:
%cd angin_topan
get_f1_eks_praproses_bencana('angin_topan')
%cd ..

[Errno 2] No such file or directory: 'angin_topan'
/workspace/Tugas Akhir/dataset/angin_topan
evaluasi pada bencana = angin_topan
/workspace/Tugas Akhir/dataset
Eksperimen pada bencana angin_topan dengan representasi title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9929577464788732
Nilai Recall = 0.8867924528301887
Nilai F1 = 0.9368770764119602
Di test dapat performa
Nilai Precision = 0.8181818181818182
Nilai Recall = 0.1956521739130435
Nilai F1 = 0.3157894736842105
########
RANDOM FOREST


/tmp/ipykernel_80140/787541875.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.15217391304347827
Nilai F1 = 0.2641509433962264
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.8920863309352518
Nilai Recall = 0.779874213836478
Nilai F1 = 0.8322147651006712
Di test dapat performa
Nilai Precision = 0.75
Nilai Recall = 0.717391304347826
Nilai F1 = 0.7333333333333332
########
MLP


/tmp/ipykernel_80140/787541875.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_80140/787541875.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7608695652173914
Nilai Recall = 0.660377358490566
Nilai F1 = 0.7070707070707071
Di test dapat performa
Nilai Precision = 0.7021276595744681
Nilai Recall = 0.717391304347826
Nilai F1 = 0.7096774193548387
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.7567698259187621
Recall = 0.5434782608695652
F1 = 0.5862667421241274
/workspace/Tugas Akhir/dataset/angin_topan


************************************************************
/workspace/Tugas Akhir/dataset
Eksperimen pada bencana angin_topan dengan representasi text_ori


...

/tmp/ipykernel_80140/787541875.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.5
Nilai Recall = 0.043478260869565216
Nilai F1 = 0.08
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.7903225806451613
Nilai Recall = 0.6163522012578616
Nilai F1 = 0.6925795053003533
Di test dapat performa
Nilai Precision = 0.5277777777777778
Nilai Recall = 0.41304347826086957
Nilai F1 = 0.4634146341463415
########
MLP


/tmp/ipykernel_80140/787541875.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_80140/787541875.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.9166666666666666
Nilai Recall = 0.06918238993710692
Nilai F1 = 0.1286549707602339
Di test dapat performa
Nilai Precision = 0.5
Nilai Recall = 0.043478260869565216
Nilai F1 = 0.08
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.4759259259259259
Recall = 0.16666666666666666
F1 = 0.20728200223178703
/workspace/Tugas Akhir/dataset/angin_topan


************************************************************
{'angin_topan': {'title_ori': {'xgb': {'prec': 0.8181818181818182, 'rec': 0.1956521739130435, 'f1': 0.3157894736842105, 'acc': 0.805}, 'rf': {'prec': 1.0, 'rec': 0.15217391304347827, 'f1': 0.2641509433962264, 'acc': 0.805}, 'lr': {'prec': 0.75, 'rec': 0.717391304347826, 'f1': 0.7333333333333332, 'acc': 0.88}, 'mlp': {'prec': 0.7021276595744681, 'rec': 0.717391304347826, 'f1': 0.7096774193548387, 'acc': 0.865}}, 'text_ori': {'xgb': {'prec': 0.8095238095238095, 'rec': 0.3695652173913043, 'f1': 0.50746268656716

/tmp/ipykernel_80140/787541875.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd banjir
get_f1_eks_praproses_bencana('banjir')
%cd ..

In [16]:
%cd erupsi
get_f1_eks_praproses_bencana('erupsi')
%cd ..


/workspace/dataset/erupsi
evaluasi pada bencana = erupsi
/workspace/dataset
Eksperimen pada bencana erupsi dengan representasi title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.8631578947368421
Nilai F1 = 0.9265536723163842
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.17647058823529413
Nilai F1 = 0.3
########
RANDOM FOREST


/tmp/ipykernel_1865/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.058823529411764705
Nilai F1 = 0.1111111111111111
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.8831168831168831
Nilai Recall = 0.7157894736842105
Nilai F1 = 0.7906976744186047
Di test dapat performa
Nilai Precision = 0.5909090909090909
Nilai Recall = 0.38235294117647056
Nilai F1 = 0.46428571428571425
########
MLP


/tmp/ipykernel_1865/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1865/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.8125
Nilai Recall = 0.4105263157894737
Nilai F1 = 0.5454545454545455
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.20588235294117646
Nilai F1 = 0.34146341463414637
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.8636363636363636
Recall = 0.2549019607843137
F1 = 0.36858304297328687
/workspace/dataset/erupsi


/workspace/dataset
Eksperimen pada bencana erupsi dengan representasi text_ori


...

/tmp/ipykernel_1865/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9894736842105263
Nilai F1 = 0.9947089947089947
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.029411764705882353
Nilai F1 = 0.05714285714285715
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.9206349206349206
Nilai Recall = 0.6105263157894737
Nilai F1 = 0.7341772151898734
Di test dapat performa
Nilai Precision = 0.6153846153846154
Nilai Recall = 0.47058823529411764
Nilai F1 = 0.5333333333333333
########
MLP


/tmp/ipykernel_1865/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1865/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.8333333333333334
Nilai Recall = 0.15789473684210525
Nilai F1 = 0.2654867256637168
Di test dapat performa
Nilai Precision = 0.7857142857142857
Nilai Recall = 0.3235294117647059
Nilai F1 = 0.45833333333333326
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.6961996336996337
Recall = 0.37254901960784315
F1 = 0.4772222222222222
/workspace/dataset/erupsi


************************************************************
{'erupsi': {'title_ori': {'xgb': {'prec': 1.0, 'rec': 0.17647058823529413, 'f1': 0.3, 'acc': 0.86}, 'rf': {'prec': 1.0, 'rec': 0.058823529411764705, 'f1': 0.1111111111111111, 'acc': 0.84}, 'lr': {'prec': 0.5909090909090909, 'rec': 0.38235294117647056, 'f1': 0.46428571428571425, 'acc': 0.85}, 'mlp': {'prec': 1.0, 'rec': 0.20588235294117646, 'f1': 0.34146341463414637, 'acc': 0.865}}, 'text_ori': {'xgb': {'prec': 0.8571428571428571, 'rec': 0.17647058823529413, 'f1': 0.29268292682926833, 'acc': 0.855},

/tmp/ipykernel_1865/1041286260.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [17]:
%cd gempa
get_f1_eks_praproses_bencana('gempa')
%cd ..


/workspace/dataset/gempa
evaluasi pada bencana = gempa
/workspace/dataset
Eksperimen pada bencana gempa dengan representasi title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.972027972027972
Nilai Recall = 0.7942857142857143
Nilai F1 = 0.8742138364779874
Di test dapat performa
Nilai Precision = 0.5
Nilai Recall = 0.08888888888888889
Nilai F1 = 0.1509433962264151
########
RANDOM FOREST


/tmp/ipykernel_1865/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.6666666666666666
Nilai Recall = 0.044444444444444446
Nilai F1 = 0.08333333333333334
########
LOGISTIK REGRESSION


/tmp/ipykernel_1865/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7971014492753623
Nilai Recall = 0.6285714285714286
Nilai F1 = 0.7028753993610224
Di test dapat performa
Nilai Precision = 0.5945945945945946
Nilai Recall = 0.4888888888888889
Nilai F1 = 0.5365853658536586
########
MLP


...

/tmp/ipykernel_1865/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.06666666666666667
Nilai F1 = 0.125
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.7672413793103449
Nilai Recall = 0.5085714285714286
Nilai F1 = 0.6116838487972509
Di test dapat performa
Nilai Precision = 0.5483870967741935
Nilai Recall = 0.37777777777777777
Nilai F1 = 0.4473684210526315
########
MLP


/tmp/ipykernel_1865/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1865/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.6818181818181818
Nilai Recall = 0.08571428571428572
Nilai F1 = 0.15228426395939088
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.044444444444444446
Nilai F1 = 0.0851063829787234
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.7066052227342551
Recall = 0.17037037037037037
F1 = 0.22877365262583627
/workspace/dataset/gempa


************************************************************
{'gempa': {'title_ori': {'xgb': {'prec': 0.5, 'rec': 0.08888888888888889, 'f1': 0.1509433962264151, 'acc': 0.775}, 'rf': {'prec': 0.6666666666666666, 'rec': 0.044444444444444446, 'f1': 0.08333333333333334, 'acc': 0.78}, 'lr': {'prec': 0.5945945945945946, 'rec': 0.4888888888888889, 'f1': 0.5365853658536586, 'acc': 0.81}, 'mlp': {'prec': 0.6285714285714286, 'rec': 0.4888888888888889, 'f1': 0.5499999999999999, 'acc': 0.82}}, 'text_ori': {'xgb': {'prec': 0.7222222222222222, 'rec': 0.28888888888888886, 'f1': 0.41269

/tmp/ipykernel_1865/1041286260.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [18]:
%cd karhutla
get_f1_eks_praproses_bencana('karhutla')
%cd ..

/workspace/dataset/karhutla
evaluasi pada bencana = karhutla
/workspace/dataset
Eksperimen pada bencana karhutla dengan representasi title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9511278195488722
Nilai Recall = 0.8724137931034482
Nilai F1 = 0.9100719424460431
Di test dapat performa
Nilai Precision = 0.7777777777777778
Nilai Recall = 0.625
Nilai F1 = 0.6930693069306931
########
RANDOM FOREST


/tmp/ipykernel_1865/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.996551724137931
Nilai Recall = 0.996551724137931
Nilai F1 = 0.996551724137931
Di test dapat performa
Nilai Precision = 0.8108108108108109
Nilai Recall = 0.5357142857142857
Nilai F1 = 0.6451612903225806
########
LOGISTIK REGRESSION


/tmp/ipykernel_1865/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.8283582089552238
Nilai Recall = 0.7655172413793103
Nilai F1 = 0.7956989247311828
Di test dapat performa
Nilai Precision = 0.6785714285714286
Nilai Recall = 0.6785714285714286
Nilai F1 = 0.6785714285714286
########
MLP


...

/tmp/ipykernel_1865/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.996551724137931
Nilai F1 = 0.9982728842832469
Di test dapat performa
Nilai Precision = 0.65625
Nilai Recall = 0.375
Nilai F1 = 0.4772727272727273
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.7915057915057915
Nilai Recall = 0.7068965517241379
Nilai F1 = 0.7468123861566484
Di test dapat performa
Nilai Precision = 0.5714285714285714
Nilai Recall = 0.5
Nilai F1 = 0.5333333333333333
########
MLP


/tmp/ipykernel_1865/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1865/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.9310344827586207
Nilai Recall = 0.09310344827586207
Nilai F1 = 0.16927899686520378
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.07142857142857142
Nilai F1 = 0.13333333333333333
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.7132034632034632
Recall = 0.33928571428571425
F1 = 0.38888888888888884
/workspace/dataset/karhutla


************************************************************
{'karhutla': {'title_ori': {'xgb': {'prec': 0.7777777777777778, 'rec': 0.625, 'f1': 0.6930693069306931, 'acc': 0.845}, 'rf': {'prec': 0.8108108108108109, 'rec': 0.5357142857142857, 'f1': 0.6451612903225806, 'acc': 0.835}, 'lr': {'prec': 0.6785714285714286, 'rec': 0.6785714285714286, 'f1': 0.6785714285714286, 'acc': 0.82}, 'mlp': {'prec': 0.7631578947368421, 'rec': 0.5178571428571429, 'f1': 0.6170212765957447, 'acc': 0.82}}, 'text_ori': {'xgb': {'prec': 0.6808510638297872, 'rec': 0.5714285714285714, 'f1': 0.6

/tmp/ipykernel_1865/1041286260.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [16]:
%cd kekeringan
get_f1_eks_praproses_bencana('kekeringan')
%cd ..


/workspace/dataset/kekeringan
evaluasi pada bencana = kekeringan
/workspace/dataset
Eksperimen pada bencana kekeringan dengan representasi title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.9932885906040269
Nilai Recall = 0.896969696969697
Nilai F1 = 0.9426751592356689
Di test dapat performa
Nilai Precision = 0.75
Nilai Recall = 0.07692307692307693
Nilai F1 = 0.13953488372093026
########
RANDOM FOREST


/tmp/ipykernel_2231603/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.0
Nilai Recall = 0.0
Nilai F1 = 0.0
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 0.9027777777777778
Nilai Recall = 0.7878787878787878
Nilai F1 = 0.8414239482200647
Di test dapat performa
Nilai Precision = 0.4772727272727273
Nilai Recall = 0.5384615384615384
Nilai F1 = 0.5060240963855421
########
MLP


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_2231603/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_2231603/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labe

Di train dapat performa 
Nilai Precision = 0.6403940886699507
Nilai Recall = 0.7878787878787878
Nilai F1 = 0.7065217391304347
Di test dapat performa
Nilai Precision = 0.44594594594594594
Nilai Recall = 0.8461538461538461
Nilai F1 = 0.584070796460177
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.5577395577395577
Recall = 0.4871794871794872
F1 = 0.4098765921888831
/workspace/dataset/kekeringan


/workspace/dataset
Eksperimen pada bencana kekeringan dengan representasi text_ori


...

/tmp/ipykernel_2231603/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.782258064516129
Nilai Recall = 0.5878787878787879
Nilai F1 = 0.671280276816609
Di test dapat performa
Nilai Precision = 0.47619047619047616
Nilai Recall = 0.5128205128205128
Nilai F1 = 0.49382716049382713
########
MLP


/tmp/ipykernel_2231603/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.7407407407407407
Nilai Recall = 0.24242424242424243
Nilai F1 = 0.365296803652968
Di test dapat performa
Nilai Precision = 0.75
Nilai Recall = 0.23076923076923078
Nilai F1 = 0.3529411764705882
########
Rata-rata nilai test dengan Classifier xgb, lr, dan mlp adalah
Precision = 0.6531746031746032
Recall = 0.34188034188034183
F1 = 0.4180585814572743
/workspace/dataset/kekeringan


************************************************************
{'kekeringan': {'title_ori': {'xgb': {'prec': 0.75, 'rec': 0.07692307692307693, 'f1': 0.13953488372093026, 'acc': 0.815}, 'rf': {'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'acc': 0.805}, 'lr': {'prec': 0.4772727272727273, 'rec': 0.5384615384615384, 'f1': 0.5060240963855421, 'acc': 0.795}, 'mlp': {'prec': 0.44594594594594594, 'rec': 0.8461538461538461, 'f1': 0.584070796460177, 'acc': 0.765}}, 'text_ori': {'xgb': {'prec': 1.0, 'rec': 0.07692307692307693, 'f1': 0.14285714285714288, 'acc': 0.82}, 'rf': {'prec': 0.857142

/tmp/ipykernel_2231603/1041286260.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd longsor
get_f1_eks_praproses_bencana('longsor')
%cd ..


/workspace/dataset/longsor
evaluasi pada bencana = longsor
/workspace/dataset
Eksperimen pada bencana longsor dengan representasi title_ori
XGBOOST
Di train dapat performa 
Nilai Precision = 0.979381443298969
Nilai Recall = 0.8755760368663594
Nilai F1 = 0.924574209245742
Di test dapat performa
Nilai Precision = 0.8695652173913043
Nilai Recall = 0.37735849056603776
Nilai F1 = 0.5263157894736843
########
RANDOM FOREST


/tmp/ipykernel_2231603/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.875
Nilai Recall = 0.2641509433962264
Nilai F1 = 0.4057971014492754
########
LOGISTIK REGRESSION


/tmp/ipykernel_2231603/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.8631578947368421
Nilai Recall = 0.7557603686635944
Nilai F1 = 0.8058968058968059
Di test dapat performa
Nilai Precision = 0.7111111111111111
Nilai Recall = 0.6037735849056604
Nilai F1 = 0.6530612244897959
########
MLP


...

/tmp/ipykernel_2231603/1041286260.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.3333333333333333
Nilai Recall = 0.32075471698113206
Nilai F1 = 0.3269230769230769
########
LOGISTIK REGRESSION


/tmp/ipykernel_2231603/1041286260.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.8628571428571429
Nilai Recall = 0.695852534562212
Nilai F1 = 0.7704081632653061
Di test dapat performa
Nilai Precision = 0.72
Nilai Recall = 0.6792452830188679
Nilai F1 = 0.6990291262135921
########
MLP


/tmp/ipykernel_2231603/1041286260.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd tsunami
get_f1_eks_praproses_bencana('tsunami')
%cd ..
